# Предобработка данных

## Импорт библиотек

In [1]:
import pandas as pd
import cv2
import os
import re
import numpy as np

from PIL import Image as PILImage

from pytesseract import Output
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
custom_config = r'--oem 3 --psm 6 '

import pyocr
pyocr.tesseract.TESSERACT_CMD = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

from IPython.display import display_html
from openpyxl import load_workbook
from io import BytesIO

from img2table.document import Image
from img2table.ocr import TesseractOCR
TesseractOCR.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
#from img2table.ocr import EasyOCR

## Загрузка и осмотр данных

In [2]:
shape_counter = 0
nan_counter = 0
columns = 10
df_list = []

for filename in os.listdir('405/'):
    if 'csv' in filename:
        path = '405/' + filename
        df = pd.read_csv(path)
        print('\n', filename)
        print(df.head())
        df_list.append(df)
        if df.shape[1] != columns:
            shape_counter += 1
        if df.isna().sum().sum() != 0:
            nan_counter += 1
print('\n', 'Число файлов где количество столбцов отличается от 10---', shape_counter)
print('\n', 'Число файлов с пропусками---', nan_counter)


 141899.csv
       ID  ID пользователя    Класс крови Дата донации  \
0   87330           141899  Цельная кровь   21.06.2019   
1  213527           141899         Плазма   29.08.2019   
2  213528           141899         Плазма   30.09.2019   
3  213529           141899         Плазма   01.11.2019   
4  213530           141899         Плазма   02.12.2019   

  Дата добавления донации   Тип донации                          Регион  \
0              18.11.2020  Безвозмездно  Россия, Татарстан, Альметьевск   
1              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
2              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
3              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   
4              16.05.2023  Безвозмездно  Россия, Татарстан, Альметьевск   

   Место стадчи Статус донации Есть справка  
0           703        Принята           Да  
1           703        Принята           Да  
2           703        Принята           Да  
3  


 243478.csv
       ID  ID пользователя    Класс крови Дата донации  \
0  215817           243478         Плазма   04.03.2016   
1  216065           243478  Цельная кровь   26.04.2016   
2  216064           243478         Плазма   01.06.2016   
3  216063           243478  Цельная кровь   04.07.2016   
4  216062           243478         Плазма   10.08.2016   

  Дата добавления донации   Тип донации                     Регион  \
0              28.05.2023  Безвозмездно  Россия, Башкортостан, Уфа   
1              29.05.2023  Безвозмездно  Россия, Башкортостан, Уфа   
2              29.05.2023  Безвозмездно  Россия, Башкортостан, Уфа   
3              29.05.2023  Безвозмездно  Россия, Башкортостан, Уфа   
4              29.05.2023  Безвозмездно  Россия, Башкортостан, Уфа   

   Место стадчи Статус донации Есть справка  
0           366        Принята           Да  
1           366        Принята           Да  
2           366        Принята           Да  
3           366        Принята   

**Чтож, неплохо. все таблицы имеют одинаковое число столбцов, пропусков нет. Жить можно**

## Добавляем город и убираем лишние столбцы

In [3]:
def region_to_city(st):
    return st.split(', ')[-1]

In [4]:
trash_columns = ['ID', 'ID пользователя', 'Дата добавления донации', 'Место стадчи', 'Статус донации', 'Есть справка', 'Регион']

for i in range(len(df_list)):
    df_list[i]['Город'] = df_list[i]['Регион'].apply(region_to_city)
    df_list[i] = df_list[i].drop(trash_columns, axis=1)

In [5]:
df_list[5].head()

,Класс крови,Дата донации,Тип донации,Город
0,Цельная кровь,03.07.2009,Безвозмездно,Ульяновск
1,Цельная кровь,16.09.2009,Безвозмездно,Ульяновск
2,Цельная кровь,16.03.2011,Безвозмездно,Ульяновск
3,Цельная кровь,01.08.2011,Безвозмездно,Ульяновск
4,Цельная кровь,24.11.2011,Безвозмездно,Ульяновск


## Функции для работы

### Препроцессинг

**Обрезка** В первом приближении - обрезаем пополам. Потому что у справки формы 405 таблица всегда в нижней половине.

**Оттенки серого**

**Удаление шумов**

**Приведение к пороговому значению** Gaussian

In [6]:
def pre(img):
    height = int(img.shape[0])
    half = int(img.shape[0] / 2)
    cut = img[half:height, :]
    gray = cv2.cvtColor(cut, cv2.COLOR_BGR2GRAY)
    noise = cv2.medianBlur(gray,1)
    thresh = cv2.adaptiveThreshold(noise,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,11)
    out = thresh
    return out

#ret,thresh3 = cv.threshold(img,127,255,cv.THRESH_TRUNC)
#ret,thresh4 = cv.threshold(img,127,255,cv.THRESH_TOZERO)

### Поиск контуров при помощи open cv (пока не работает)

In [7]:
def cont(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    img_contours = np.uint8(np.zeros((img.shape[0],img.shape[1])))
    #for cnt in contours:
        #approx = cv2.approxPolyDP(cnt, 0.009 * cv2.arcLength(cnt, True), True)  
    #all_contours = cv2.drawContours(img_contours, contours, -1, (255,255,255), 1)
    #cell = []
    #for contour in contours:
        #if contour.shape[0] > 50:
            #cell.append(contour)
    return cv2.drawContours(img_contours, contours, -1, (0, 0, 255), 5) 
#cl = np.array(cell).reshape((-1,1,2)).astype(np.int32)

### Выравнивание изображения (для изображений с искаженной перспективой работает некорректно)

In [8]:
def deskew(img):
    coords = np.column_stack(np.where(img > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

### Поиск координат ячеек таблиц с помощью img2table

In [9]:
def search(path):
    img = Image(src=path)
    extracted_tables = img.extract_tables()
    table_img = (cv2.imread(path))
    find_cell = []
    for table in extracted_tables:
        for row in table.content.values():
            for cell in row:
                cv2.rectangle(table_img, (cell.bbox.x1, cell.bbox.y1), (cell.bbox.x2, cell.bbox.y2), (255, 0, 0), 2)
                rect = [cell.bbox.y1, cell.bbox.y2, cell.bbox.x1, cell.bbox.x2]
                find_cell.append(rect)
    return find_cell
    #PILImage.fromarray(table_img)

### Распознавание числовых значений и дат

In [10]:
def numbers(cells, path):
    cfg = r'--oem 3 --psm 7 outputbase digits'
    nums = []
    img = (cv2.imread(path))
    for place in cells:
        img_part = img[place[0]:place[1],place[2]:place[3]]
        nums.append(pytesseract.image_to_string(img_part, lang='rus', config=cfg))
    return nums

### Распознавание всего остального

In [11]:
def words(cells, path):
    cfg = r'--oem 3 --psm 7'
    wds = []
    img = (cv2.imread(path))
    for place in cells:
        img_part = img[place[0]:place[1],place[2]:place[3]]
        wds.append(pytesseract.image_to_string(img_part, lang='rus', config=cfg))
    return wds

### Объединение распознанных данных

In [12]:
def join_data(n, w):
    for i in range(len(n)):
        if n[i] == '':
            n[i] = w[i]
    return n

### Формирование выходных значений (пробная версия, надо переделывать)

In [13]:
def ser(list):
    date = []
    volume = []
    index_d = []
    c_d = 0
    index_v = []
    c_v = 0
    for i in range(len(list)):
        if len(list[i].split('.')) > 2:
            date.append(list[i])
            c_d += 1
            index_d.append(c_d)
        elif re.search('\d', list[i]) and len(list[i]) > 3 and ' ' not in list[i]:
            volume.append(list[i])
            c_v += 1
            index_v.append(c_v)
        
    return pd.Series(date, index_d, dtype=pd.StringDtype()), pd.Series(volume, index_v, dtype=pd.StringDtype())
        

# Попытки что-то распознать

## Загрузка и предобработка изображений

**Взято плохое изображение, изображение среднего качества и высокого качества**

In [14]:
bad = cv2.imread('405/141899 .jpg')
good = cv2.imread('405/245365 .jpg')
third = cv2.imread('405/231820 .jpg')

good_pre = pre(good)
bad_pre = pre(bad)
third_pre = pre(third)

cv2.imwrite('good.png', good_pre)
cv2.imwrite('bad.png', bad_pre)
cv2.imwrite('third.png', third_pre)

cv2.imshow('good after', good_pre)
cv2.imshow('good before', good)
cv2.imshow('bad after', bad_pre)
cv2.imshow('bad before', bad)
cv2.imshow('third after', third_pre)
cv2.imshow('third before', third)

im_path_list = ['good.png', 'bad.png', 'third.png']

cv2.waitKey()
cv2.destroyAllWindows()

## Распознавание

In [15]:
raw_out = []
for pth in im_path_list:
    n = numbers(search(pth), pth)
    w = words(search(pth), pth)
    raw_out.append(join_data(n, w))

In [16]:
for out in raw_out:
    data, volume = ser(out)
    print(data.head(), '\n', volume.head())

1    14.01.2009

2    16.06.2017

3    16.06.2017

4    20.04.2019

5    14.07.2009

dtype: string 
 1     370

2     450

3    450.

4    450.

5    450.

dtype: string
Series([], dtype: string) 
 Series([], dtype: string)
1     14.02.2006

2    15.07.2016.

3     04.08.2018

4     11.10.2016

5     26.12.2018

dtype: string 
 1     420

2     450

3     450

4    .350

5     68.

dtype: string


### img2table "из коробки" так себе распознает

In [17]:
#ocr = TesseractOCR(n_threads=1, lang="rus")
#ocr.config = custom_config
#extracted_tables = img.extract_tables(ocr=ocr,
                                      #implicit_rows=False,
                                      #borderless_tables=False,
                                      #min_confidence=25)
#table = extracted_tables.pop()
#display_html(table.html_repr(), raw=True)

# Расчет accuracy